# mse budget

Developent of mse

$\frac{1}{2} \hat{\frac{\delta {h ^ \prime} ^2}{\delta t}} = \hat{h^ \prime} \ \text{NetLW} ^ \prime + \hat{h^ \prime} \ \text{SEF} ^ \prime + \hat{h^ \prime} \ \text{NetSW} ^ \prime - \hat{h^ \prime} \ \nabla_{h}\cdot \hat{\overrightarrow{u}h}$


In [ ]:
import xarray as xr

import numpy as np
import skimage.measure as skm
import scipy
import pandas as pd

import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeat

import os
from funcs.vars.myPlots import *

In [ ]:
model='MPI-ESM1-2-HR'
experiment_id='historical'

In [ ]:
# mse
folder = '/Users/cbla0002/Documents/data/cmip6/' + model
fileName = model + '_mse_' + experiment_id + '.nc'
path = folder + '/' + fileName
ds = xr.open_dataset(path)
mse = ds.mse
mse_sMean = mse.mean(dim=('lat','lon'))
mse_anom = mse - mse_sMean
mse_anomSqrd = mse_anom**2
mse_var = mse_anomSqrd.mean(dim=('lat','lon'))
dmse = mse_anomSqrd.isel(time=slice(1,None)).data-mse_anomSqrd.isel(time=slice(0,-1)).data
dmse = xr.DataArray(
    data=dmse,
    dims=['time', 'lat', 'lon'],
    coords={'time': mse_anomSqrd.time.data[0:-1], 'lat': mse_anomSqrd.lat.data, 'lon': mse_anomSqrd.lon.data},
    attrs={'units':''}
    )
dmse_var = dmse.mean(dim=('lat','lon'))/2


del ds, mse_sMean, mse_anomSqrd, mse_var, dmse_var
#mse, 
#mse_anom,
#dmse

<xarray.DataArray (time: 10956)>
array([ 1.92450059e+14,  1.14706904e+15,  5.52745345e+14, ...,
        1.08281717e+13, -9.92127629e+14,  3.84955787e+13])
Coordinates:
  * time     (time) datetime64[ns] 1970-01-01T12:00:00 ... 1999-12-30T12:00:00

In [ ]:
# LW
folder = '/Users/cbla0002/Documents/data/cmip6/' + model
fileName = model + '_netlw_' + experiment_id + '.nc'
path = folder + '/' + fileName
ds = xr.open_dataset(path)
netlw = ds.netlw
netlw_sMean = netlw.mean(dim=('lat','lon'))
netlw_anom = netlw - netlw_sMean
netlw_corr = mse_anom.data * netlw_anom.data
netlw_corr = xr.DataArray(
    data=netlw_corr[0:-1],
    dims=['time', 'lat', 'lon'],
    coords={'time': mse_anom.time.data[0:-1], 'lat': mse_anom.lat.data, 'lon': mse_anom.lon.data},
    attrs={'units':''}
    )

del ds, netlw_sMean, netlw_anom
#netlw, 

<xarray.DataArray 'netlw' (time: 10957, lat: 64, lon: 384)>
array([[[-55.258682 , -54.789444 , -12.731827 , ..., -36.010635 ,
         -49.06044  , -61.50917  ],
        [-46.547745 , -28.12294  ,  -6.5868073, ..., -22.307999 ,
         -51.057022 , -51.46962  ],
        [-68.0038   , -67.474014 , -74.62196  , ..., -65.18593  ,
         -65.84364  , -68.487686 ],
        ...,
        [ 58.400497 ,  60.463486 ,  53.94493  , ...,  55.71837  ,
          48.807724 ,  50.41661  ],
        [ 54.7213   ,  60.588974 ,  58.839462 , ...,  62.144638 ,
          55.270126 ,  51.54161  ],
        [ 52.034286 ,  55.024033 ,  52.85753  , ...,  65.33995  ,
          60.256943 ,  51.50255  ]],

       [[-45.548325 , -57.507797 , -64.36815  , ..., -24.698715 ,
         -48.45262  , -70.32518  ],
        [-52.952133 , -57.189438 , -28.52977  , ...,   2.4868622,
         -53.27098  , -32.970688 ],
        [-34.767075 , -40.709946 , -71.18114  , ...,  -3.0565948,
         -45.25096  , -32.357407 ],
...
        [ 74.35031  ,  72.78635  ,  65.99533  , ...,  76.403534 ,
          73.2092   ,  75.763885 ],
        [ 70.84592  ,  68.66574  ,  63.849335 , ...,  77.494354 ,
          80.83957  ,  80.67746  ],
        [ 63.08322  ,  63.355682 ,  60.355194 , ...,  80.25363  ,
          76.21799  ,  70.34592  ]],

       [[-39.283127 , -35.402298 , -35.10054  , ..., -31.97261  ,
         -37.366165 , -40.422318 ],
        [-28.581497 , -36.780716 , -53.903244 , ..., -27.298294 ,
         -31.437943 , -33.940872 ],
        [-13.873001 , -63.215744 , -61.431076 , ..., -32.289505 ,
         -32.570755 , -36.77974  ],
        ...,
        [ 31.501541 ,  32.667557 ,  32.87361  , ...,  31.39656  ,
          28.70076  ,  28.179764 ],
        [ 26.878983 ,  30.83406  ,  32.916092 , ...,  34.01326  ,
          30.132889 ,  26.74324  ],
        [ 23.585526 ,  25.557693 ,  29.833572 , ...,  32.009842 ,
          28.786697 ,  24.752518 ]]], dtype=float32)
Coordinates:
  * lat      (lat) float64 -29.45 -28.52 -27.58 -26.65 ... 27.58 28.52 29.45
  * lon      (lon) float64 0.0 0.9375 1.875 2.812 ... 356.2 357.2 358.1 359.1
  * time     (time) datetime64[ns] 1970-01-01 1970-01-02 ... 1999-12-31

In [ ]:
# SEF
folder = '/Users/cbla0002/Documents/data/cmip6/' + model
fileName = model + '_netsef_' + experiment_id + '.nc'
path = folder + '/' + fileName
ds = xr.open_dataset(path)
netsef = ds.netsef
netsef_sMean = netsef.mean(dim=('lat','lon'))
netsef_anom = netsef - netsef_sMean
netsef_corr = mse_anom.data * netsef_anom.data
netsef_corr = xr.DataArray(
    data=netsef_corr[0:-1],
    dims=['time', 'lat', 'lon'],
    coords={'time': mse_anom.time.data[0:-1], 'lat': mse_anom.lat.data, 'lon': mse_anom.lon.data},
    attrs={'units':''}
    )

del ds, netsef_sMean, netsef_anom
#netsef, 

In [ ]:
# SW
folder = '/Users/cbla0002/Documents/data/cmip6/' + model
fileName = model + '_netsw_' + experiment_id + '.nc'
path = folder + '/' + fileName
ds = xr.open_dataset(path)
netsw = ds.netsw
netsw_sMean = netsw.mean(dim=('lat','lon'))
netsw_anom = netsw - netsw_sMean
netsw_corr = mse_anom.data * netsw_anom.data
netsw_corr = xr.DataArray(
    data=netsw_corr[0:-1],
    dims=['time', 'lat', 'lon'],
    coords={'time': mse_anom.time.data[0:-1], 'lat': mse_anom.lat.data, 'lon': mse_anom.lon.data},
    attrs={'units':''}
    )

del ds, netsw_sMean, netsw_anom
#netsw, 

In [ ]:
# ADV
adv = (netlw_corr + netsw_corr + netsef_corr) - dmse

del netlw_corr, netsw_corr, netsef_corr, dmse

In [ ]:
# adv_anom = adv - adv.mean(dim = ('lat', 'lon'))

In [ ]:
# adv_corr = mse_anom * adv_anom 

In [ ]:
# adv_corr = xr.DataArray(
#     data=adv_corr,
#     dims=['time', 'lat', 'lon'],
#     coords={'time': mse_anom.time.data[0:-1], 'lat': mse_anom.lat.data, 'lon': mse_anom.lon.data},
#     attrs={'units':''}
#     )